<a href="https://colab.research.google.com/github/LucasAlbFar/analise_dados_hepatite_brasil/blob/main/notebooks/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.ticker as ticker
import numpy as np
import statistics

plt.rc('figure', figsize=(20,7))

In [101]:
url = 'https://raw.githubusercontent.com/LucasAlbFar/analise_dados_hepatite_brasil/main/dados/doses_aplicadas_por_ano_1994_2021.csv'

In [103]:
doses_aplicadas = pd.read_csv(url, encoding="ISO-8859-1", sep=";", thousands='.', decimal=',')
doses_aplicadas = doses_aplicadas.drop(['1994'], axis=1) # removendo coluna do ano de 1994

total_por_ano = doses_aplicadas.iloc[-1:].T     # salvando o total por ano 
doses_aplicadas = doses_aplicadas.iloc[:-1 , :] # removendo última linha 

doses_aplicadas['Unidade da Federação'] = [(estado[3:]) for estado in doses_aplicadas['Unidade da Federação']] # renomeando coluna UNIDADE DA FEDEREACAO
doses_aplicadas = doses_aplicadas.set_index('Unidade da Federação')                                            # UF's como indíce

total_por_uf =  doses_aplicadas.iloc[: , -1:]   # salvando total por uf
doses_aplicadas = doses_aplicadas.iloc[: , :-2] # removendo coluna total e ano de 2021

doses_aplicadas

,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Unidade da Federação,,,,,,,,,,,,,,,,,,,,,,,,,,
Rondônia,207459,144735,48204,284513,242985,206296,292553,168095,144339,128169,163340,150128,130536,158254,152052,142750,156774,145596,139752,157942,158399,129642,99495,92868,92366,71858
Acre,143845,71667,13460,67947,1305380,98465,102509,84525,84971,73776,75675,67871,83924,95347,105503,102222,117360,106904,72409,50955,53746,39248,45873,43463,45925,30880
Amazonas,325411,299571,64637,346619,496898,434945,481011,446048,396598,326329,356846,350897,374693,361393,358219,343181,356573,337960,291348,326899,297631,181756,245458,271990,273327,210123
Roraima,51715,65149,22290,130516,94311,106533,67228,68692,69055,58151,80040,71139,69688,77781,67499,70721,68763,62036,58081,70284,70289,42897,86875,90442,77269,55667
Pará,304709,310588,47329,303487,1565678,801902,1090151,1338385,1298510,834766,796893,746010,753835,717106,655370,661978,707089,714286,705853,568474,444935,267492,432441,389415,407324,331734
Amapá,26320,48448,5891,71395,138089,109414,154072,109364,91928,66098,65280,61688,60916,77572,70762,86950,77110,72390,53680,54348,60370,47575,50889,43991,45813,27905
Tocantins,0,119700,36653,342615,367434,298045,267479,222614,203614,142491,150169,128696,122637,125628,106307,118082,126214,110612,129179,139990,103585,59184,90441,80323,83442,68392
Maranhão,18717,21086,20955,217434,1752105,1303390,999999,831603,744063,656124,663516,642167,619627,630308,601414,693618,797951,751562,692688,647301,624073,339966,505626,433489,405860,306017
Piauí,2524,673,250,4427,187320,273151,360373,282231,656926,405909,322608,277908,287079,233866,238804,242285,313273,272943,270824,250057,180644,106730,160172,145106,155048,128168
